In [1]:
import re
import time
import datetime
import requests
import pandas as pd
import multiprocessing
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

from Script.Function.funciones import movies_for_category, categories, print_process, movies, series_informacion, desempaquetado, generador_dummies

# -------------- Parametros --------------
ruta_base = r"Bases_datos/"

# ----------------------------------------

In [55]:
print_process("Comienzo del Notebook 'Scraping'")

|-----------------------Comienzo del Notebook 'Scraping'-----------------------|


In [56]:
categorias = categories()

|---------------------Categorias capturadas correctamente.---------------------|
|-----------------------Categorias demoro 9.15 segundos-----------------------|


In [57]:

# Obtener la cantidad de núcleos disponibles en el sistema
num_nucleos = multiprocessing.cpu_count()

# Crear un diccionario para almacenar los resultados
resultados_dict = {}

categorias_comienzo = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies_for_category, categorias))
    
    # Asignar los resultados al diccionario
    for categoria, resultado in zip(categorias, resultados):
        resultados_dict[categoria] = resultado

peliculas_total = []

for x,i in resultados_dict.items():
    for e in i:
        peliculas_total.append(e)

peliculas_total = set(peliculas_total)

categorias_fin = time.time()

print_process(f"La extraccion total de las categorias tubo una demora de {round((categorias_fin-categorias_comienzo),2)} segundos")


|Todo el contenido de la categoria 'history' fue capturado correctamente, demoro 21.72 segundos|
|Todo el contenido de la categoria 'crime' fue capturado correctamente, demoro 25.92 segundos|
|Todo el contenido de la categoria 'family' fue capturado correctamente, demoro 26.22 segundos|
|Todo el contenido de la categoria 'biography' fue capturado correctamente, demoro 28.48 segundos|
|Todo el contenido de la categoria 'adventure' fue capturado correctamente, demoro 30.47 segundos|
|Todo el contenido de la categoria 'animation' fue capturado correctamente, demoro 32.13 segundos|
|Todo el contenido de la categoria 'action' fue capturado correctamente, demoro 35.28 segundos|
|Todo el contenido de la categoria 'fantasy' fue capturado correctamente, demoro 38.82 segundos|
|Todo el contenido de la categoria 'music' fue capturado correctamente, demoro 14.57 segundos|
|Todo el contenido de la categoria 'musical' fue capturado correctamente, demoro 10.51 segundos|
|Todo el contenido de la categ

In [58]:
peliculas_total = [x for x in peliculas_total if "/content/" in x]

In [59]:
print_process("Comienzo del proceso de extraccion de informacion de cada iteracion")

|-----Comienzo del proceso de extraccion de informacion de cada iteracion-----|


In [60]:
# Crear un diccionario para almacenar los resultados
todo = {"titulo": [],"idioma" : [], "subtitulos" : [], "ano": [], "duracion_minutos": [], "categorias": [], "sinopsis": [], "tipo": [], "link": []}

comienzo_movies = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies, peliculas_total))

    try:
        for resultado in resultados:
            for clave, valor in resultado.items():
                todo[clave].append(valor)
    except:
        print()

fin_movies = time.time()

print_process(f"El proceso de captura de la informacion de todo el contenido fue exsitoso, y demoro {round((fin_movies - comienzo_movies) / 60,2)} minutos")


Error al cargar la url https://filmzie.com/content/game-of-love-2022 en el intento 1: 404
Error al cargar la url https://filmzie.com/content/12-days-with-god-2019 en el intento 1: 404
Error al cargar la url https://filmzie.com/content/to-remember-me-by-2019 en el intento 1: 404
Error al cargar la url https://filmzie.com/content/summer-snow-2014 en el intento 1: 404
|El proceso de captura de la informacion de todo el contenido fue exsitoso, y demoro 6.49 minutos|


In [61]:
df = pd.DataFrame(todo)

In [62]:

df.to_csv(f"{ruta_base}Peliculas_Series_Completo_Respaldo.csv" , index=False)

In [2]:
df = pd.read_csv(f"{ruta_base}Peliculas_Series_Completo_Respaldo.csv")

In [3]:
peliculas = df.loc[df["tipo"] == "Pelicula"].reset_index(drop=True)
series = df.loc[df["tipo"] == "Serie"].reset_index(drop=True)

In [4]:

peliculas["duracion_minutos"] = peliculas["duracion_minutos"].str.replace(" min","").astype(int)
peliculas.loc[peliculas["idioma"] == 'Non linguistic content', "idioma"] = "No contiene idioma"
peliculas["subtitulos"] = peliculas["subtitulos"].fillna("No contiene subtitulos")


In [5]:
peliculas.head()

,titulo,idioma,subtitulos,ano,duracion_minutos,categorias,sinopsis,tipo,link
0,Nao Yik,Multiple languages,No contiene subtitulos,2011,58,"Documentary, European Production",Experience a captivating documentary journey t...,Pelicula,https://filmzie.com/content/nao-yik
1,Nick Whitmer: Always Hungry,English,English,2022,39,Comedy,"In his first standup special, comedian Nick Wh...",Pelicula,https://filmzie.com/content/nick-whitmer-alway...
2,Life off Grid,English,English,2016,85,"Documentary, News",A two-year journey exploring the lives of Cana...,Pelicula,https://filmzie.com/content/life-off-grid-2016
3,El Crazy Che,Spanish,English,2015,86,"Documentary, Animation, Biography, European Pr...",Witness the astounding true-life journey of an...,Pelicula,https://filmzie.com/content/el-crazy-che-2015
4,My Year With Helen,English,English,2017,93,Documentary,Embark on a journey with ex-New Zealand PM Hel...,Pelicula,https://filmzie.com/content/my-year-with-helen...


In [6]:
peliculas.to_csv(f"{ruta_base}Peliculas.csv",index=False)

In [7]:
peliculas_dummies = generador_dummies(peliculas)

In [8]:
# Exportamos base de datos de Peliculas con dummies
peliculas_dummies.to_csv(f"{ruta_base}Peliculas_dummies.csv",index=False)

peliculas_dummies.head()

,titulo,idioma,subtitulos,ano,duracion_minutos,sinopsis,tipo,link,Adventure,Sport,...,Documentary,News,Comedy,War,Biography,Action,European Production,History,Reality TV,Thriller
0,Nao Yik,Multiple languages,No contiene subtitulos,2011,58,Experience a captivating documentary journey t...,Pelicula,https://filmzie.com/content/nao-yik,0,0,...,1,0,0,0,0,0,1,0,0,0
1,Nick Whitmer: Always Hungry,English,English,2022,39,"In his first standup special, comedian Nick Wh...",Pelicula,https://filmzie.com/content/nick-whitmer-alway...,0,0,...,0,0,1,0,0,0,0,0,0,0
2,Life off Grid,English,English,2016,85,A two-year journey exploring the lives of Cana...,Pelicula,https://filmzie.com/content/life-off-grid-2016,0,0,...,1,1,0,0,0,0,0,0,0,0
3,El Crazy Che,Spanish,English,2015,86,Witness the astounding true-life journey of an...,Pelicula,https://filmzie.com/content/el-crazy-che-2015,0,0,...,1,0,0,0,1,0,1,0,0,0
4,My Year With Helen,English,English,2017,93,Embark on a journey with ex-New Zealand PM Hel...,Pelicula,https://filmzie.com/content/my-year-with-helen...,0,0,...,1,0,0,0,0,0,0,0,0,0


In [9]:
series.drop(columns=['duracion_minutos'], inplace=True)
series

,titulo,idioma,subtitulos,ano,categorias,sinopsis,tipo,link
0,By Night,English,English,2016,"Drama, Horror, Mystery",Experience anthology amplified! Short episodes...,Serie,https://filmzie.com/content/by-night-2016
1,Rescue Warriors,English,English,2005,Documentary,Discover the rigorous journey of USAF Pararesc...,Serie,https://filmzie.com/content/rescue-warriors-2005
2,Heroes Manufactured: Creators Unleashed,English,English,2020,Reality TV,A reality series following North America's top...,Serie,https://filmzie.com/content/heroes-manufacture...
3,Alaska Outdoors Magazine,English,NaN,2023,Documentary,The Best of Alaska Outdoors showcases action-p...,Serie,https://filmzie.com/content/alaska-outdoors-ma...
4,Chemistry Series,English,NaN,1983,Documentary,Explore the world of chemistry through these i...,Serie,https://filmzie.com/content/chemistry-series-1983
...,...,...,...,...,...,...,...,...
94,Animal Armory,English,NaN,2016,"Documentary, European Production",The lives of animals are not peaceful. They mu...,Serie,https://filmzie.com/content/animal-armory-2016
95,Before the Fame,English,English,2018,"Biography, Documentary","""Before the Fame"" takes us down memory lane to...",Serie,https://filmzie.com/content/before-the-fame-2018
96,Dynamo,English,English,2012,Sci-Fi,"Explore a cyberpunk web series with sci-fi, fa...",Serie,https://filmzie.com/content/dynamo-2012
97,The Special Seer,English,English,2019,Animation,There are worlds in battle waiting for a savio...,Serie,https://filmzie.com/content/the-special-seer-2019


In [10]:
links_series = list(series["link"].values)
links_series

['https://filmzie.com/content/by-night-2016',
 'https://filmzie.com/content/rescue-warriors-2005',
 'https://filmzie.com/content/heroes-manufactured-creators-unleashed-2020',
 'https://filmzie.com/content/alaska-outdoors-magazine-2023',
 'https://filmzie.com/content/chemistry-series-1983',
 'https://filmzie.com/content/we-are-all-human-2020',
 'https://filmzie.com/content/celebrity-fish-n-fire-2016',
 'https://filmzie.com/content/bommi--friends-2016',
 'https://filmzie.com/content/in-search-of-walt-whitman-2020',
 'https://filmzie.com/content/missing-kenley-2021',
 'https://filmzie.com/content/new-zealand-from-above-2012',
 'https://filmzie.com/content/chop-socky-boom-2012',
 'https://filmzie.com/content/an-animal-fact-for-you-2021',
 'https://filmzie.com/content/ghost-dimension-2016',
 'https://filmzie.com/content/dark-passages-2021',
 'https://filmzie.com/content/one-world-2021',
 'https://filmzie.com/content/conquering-northern-china-2017',
 'https://filmzie.com/content/rescue-vet-2

In [11]:
procesar = links_series

# Obtener la cantidad de núcleos disponibles en el sistema
num_nucleos = multiprocessing.cpu_count()

# Crear un diccionario para almacenar los resultados
resultados_dict = {"titulo": [], "temporadas": [], "capitulos": [], "sinopsis" : [], "duracion_minutos" : []}
# diccionario = {"titulo" : alm_3[0], "temporada" : alm_3[1], "capitulos" : alm_3[2], "sinopsis" : alm_3[3], "duracion_minutos" : alm_3[4]}
series_comienzo = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados_temporales = list(executor.map(series_informacion, procesar))

# Separar los resultados temporales en listas individuales
titulos, temporadas, capitulos, sinopsis, duracion_minutos = zip(*resultados_temporales)

# Agregar los resultados a los diccionarios
resultados_dict["titulo"].extend(titulos)
resultados_dict["temporadas"].extend(temporadas)
resultados_dict["capitulos"].extend(capitulos)
resultados_dict["sinopsis"].extend(sinopsis)
resultados_dict["duracion_minutos"].extend(duracion_minutos)

series_fin = time.time()

print_process(f"La extraccion total de informacion de cada serie tuvo una demora de {round((series_fin-series_comienzo)/ 60, 2) } minutos")

Tiempo total de Bommi & Friends: 29.0 segundos
Tiempo total de Alaska Outdoors Magazine: 72.13 segundos
Tiempo total de New Zealand from Above: 90.29 segundos
Tiempo total de In Search of Walt Whitman: 92.03 segundos
Tiempo total de We Are All Human: 92.49 segundos
Tiempo total de Rescue Warriors: 94.75 segundos
Tiempo total de Celebrity Fish N Fire: 97.88 segundos
Tiempo total de Missing Kenley: 102.82 segundos
Tiempo total de Chemistry Series: 108.26 segundos
Tiempo total de Chop Socky Boom: 108.48 segundos
Tiempo total de Heroes Manufactured: Creators Unleashed: 109.3 segundos
Tiempo total de By Night: 109.41 segundos
Tiempo total de One World: 49.34 segundos
Tiempo total de Dark Passages: 51.45 segundos
Tiempo total de Ghost Dimension: 74.77 segundos
Tiempo total de An Animal Fact For You: 123.98 segundos
Tiempo total de Inner Worlds: The Series: 68.48 segundos
Tiempo total de Conquering Northern China: 75.43 segundos
Tiempo total de The Inflight Food Trip: 69.85 segundos
Tiempo to

In [12]:
df_2 = pd.DataFrame(desempaquetado(resultados_dict))
df_2.head()

,titulo,temporadas,capitulos,sinopsis,duracion_minutos
0,By Night,Season 1,1. Airborne,A chance encounter in a bar sends a pilot into...,15
1,Rescue Warriors,Season 1,"1. Crawl, Walk, Run",Before Pararescue trainees enter the intense t...,47
2,Rescue Warriors,Season 1,2. Pay Now or Pay Later,Before Pararescue trainees enter the intense t...,47
3,Rescue Warriors,Season 1,3. Never Quit,Before Pararescue trainees enter the intense t...,48
4,Rescue Warriors,Season 1,4. That Others May Live,Dive into the heart of intense two-year Parare...,48


In [13]:
series_descripcion = pd.merge(series,df_2, on="titulo", how="inner")
series_descripcion.head

<bound method NDFrame.head of                titulo    idioma subtitulos   ano              categorias  \
0            By Night   English    English  2016  Drama, Horror, Mystery   
1     Rescue Warriors   English    English  2005             Documentary   
2     Rescue Warriors   English    English  2005             Documentary   
3     Rescue Warriors   English    English  2005             Documentary   
4     Rescue Warriors   English    English  2005             Documentary   
...               ...       ...        ...   ...                     ...   
996     Benza English  Japanese    English  2020                  Comedy   
997     Benza English  Japanese    English  2020                  Comedy   
998     Benza English  Japanese    English  2020                  Comedy   
999     Benza English  Japanese    English  2020                  Comedy   
1000    Benza English  Japanese    English  2020                  Comedy   

                                             sinopsis_x  